In [1]:
import itertools as it
from itertools import chain
from sage.combinat.shuffle import ShuffleProduct

SPARSE = True

s = SymmetricFunctions(QQ).schur()  #Schur basis for symmetric functions
p = SymmetricFunctions(QQ).power()  #Power-sum symmetric functions

### Converting formal symbols (cells and permutations) to actual vectors and matrices:

def sign(s):                   #return the sign of the permutation (-1)^s
    return Permutation(s).sign()

def rho(s):    #return the matrix associated with a permutation of the configuration points: sign(s)*rho(s)^(-1)
    if len(s)<n:
        s.append(n)
    s = Permutation(s) #convert to permutation
    return s.sign()*rep.representation_matrix(s.inverse())


def vector_form(pair): #convert a formal chain into an actual vector
    [cell,permutation] = pair
    #model a direct sum of N copies of a space of matrices by stacking them one on top of the other
    if cell[-1] == n:
        rows = topcells
    else: #cell[-1] == n-1
        rows = botcells

    mat = matrix(d*len(rows),d, sparse = SPARSE)      #matrix of zeros
    offset = rows.index(cell) #linearly order the cells, find position of current cell
    mat[d*offset: d*(offset+1)] = rho(permutation)    #insert "permutation" in the right place
    return mat

### Operations on cells: drop index (for differential), flips, swaps, left/right transvections, and the full cycle

def drop(k, cell):                    #dropping the k-th entry results in moving it to the last element
    new_cell = cell.copy()
    for j in range(1,len(cell)):
        if cell[j] >= k :
            new_cell[j] -= 1
    new_ordering = list(chain(range(1,k),range(k+1,cell[-1]+1), [k]))
    return vector_form( [new_cell, new_ordering] )

def diffl(cell): # the differential applied to a cell, given in vector form
    bdry = matrix(d*len(botcells),d, sparse = SPARSE) # initialize the zero vector form.
    for i in range(1,g+1):  #consider the i-th arc
        if cell[i] - cell[i-1] > 1:        #arcs with one point or less do not contribute to the boundary
            bdry += drop(cell[i-1]+1,cell) - drop(cell[i],cell) # differential for partition p
    return bdry

def double_matrix(cells,mult): #construct block diagonal matrix, with each block a doubling operator that doubles the i-th arc mult[i]-times
    mat = matrix(d*len(cells),0, sparse = True)
    for cell in cells:
        block = matrix.identity(d)      #initialize a new block
        for j in range(g): # for doubling of the j-th arc
            if mult[j] == 0: # in case of no doubling, proceed to next arc
                continue
            temp = matrix(ZZ,d) # sum over shuffles will be added here
            for k in range(cell[j],cell[j+1]+1): #break the j-th arc into "initial" and "terminal" parts
                #shuffle points from latter part into the former
                for shuffle in ShuffleProduct(range(cell[j]+1,k+1),range(k+1,cell[j+1]+1)):
                    new_ordering =  list(chain( range(1,cell[j]+1), shuffle, range(cell[j+1]+1,cell[-1]+1) ))
                    temp += rho(new_ordering) #add the permutation to the lot
            # now term is the doubling action on the j-th arc. Need to apply it mult[j] times into the block
            block *= temp^mult[j]
        #now block is ready, need to augment the total matrix by the column with "block" on the diagonal term
        col = matrix(d*len(cells),d , sparse = True) #column of zeros

        offset = cells.index(cell) #linearly order the cells, find position of current cell
        col[d*offset: d*(offset+1)] = block    #insert "block" in the right place
        mat = mat.augment(col)
    return mat

def conf_trace(num_points, genus, evalues, codim = 1, partitions = [], only_rank = False, Show_Progress = False):
    #Returns the character of isometry invariants of homology in codimension=*codim*, restricted to optional set of partitions
    #(default is all partitions and codimension 1)
    #Option to compute only the rank of isotypical components with the parameter only_rank
    #Option to print out partial calculations as Schur functions - use to know which partitions are crashing.
    
    global n #number of points
    global g #genus
    global topcells
    global botcells
    global rep #current representation
    global d #dimension of current representation

    n = num_points
    g = genus

    #a cell is indexed by 12...i_1 | i_1+1... i_2 | ... | i_{g-1}+1...n
    topcells = [list(chain([0],l,[n])) for l in it.combinations_with_replacement(range(0,n+1),g-1)]
    #a cell is indexed by 12...i_1 | i_1+1... i_2 | ... | i_{g-1}+1...n-1
    botcells = [list(chain([0],l,[n-1])) for l in it.combinations_with_replacement(range(0,n),g-1)]

    eigenvalues = {} #dictionary recording trace of a matrix with evalues [l_1,l_2,...]

    if len(partitions) == 0:
        partitions = Partitions(n).list()
    else:
        partitions = [Partition(p) for p in partitions]

    for p in partitions:                 #run over the partitions~irreducibles

        if Show_Progress:
            print("For partition ", p," -")
        eigenvalues[p] = []

        rep = SymmetricGroupRepresentation(p, cache_matrices = False)   #make the Specht module associated with the partition - this assigns a matrix to every permutation
        d = rho(range(1,n+1)).ncols()           #dimension of the representation

        rel = matrix(d*len(botcells),0 , sparse = SPARSE)     #start constructing relations matrix.
        #It starts with the differential, then continues with Id-isoms, so that kernel is top homology invariants (and coker is bottom homology coinvariants)

        # compute differential: start with matrix with 0 columns and later augment with all vector forms
        if Show_Progress: print("***Computing differential:...",end="",flush = True)
        for cell in topcells:
            rel = rel.augment(diffl(cell))
        if Show_Progress: print("..............done!***",flush = True)
            
        if only_rank:
            if Show_Progress:
                print("***Computing rank:....", end = "")
            rank = rel.rank()
            if codim == 0:
                result = d*len(topcells)-rank
            if codim == 1:
                result = d*len(botcells)-rank
                
            if Show_Progress: print(f"......................done! (rank {result})***",flush = True)
            eigenvalues[p] = [([1]*genus, result)]
            
            continue

        #now rel is the differential

        if codim == 0:
            if Show_Progress: print("***Computing kernel:...",end="",flush = True)
            rel = rel.transpose().change_ring(QQ) #transposing since "kernel()" function computes the left kernel
            H = rel.kernel() #cycles of top-chains = top-homology
            if Show_Progress: print(f"......................done! (dimension {H.dimension()})***",flush = True)
            del rel #don't need differential anymore
            for evalue in evalues:
                trace = double_matrix(topcells,evalue).transpose().change_ring(QQ).restrict(H).trace()
                evalue = [2^d for d in evalue]
                eigenvalues[p] += [(evalue,trace)]
                if Show_Progress:
                    print(f"Trace of matrix with e-values {evalue} is: {trace}")

        else: #codim = 1
            if Show_Progress: print("***Computing cokernel:...",end="",flush = True)
            rel = rel.change_ring(QQ)
            H = rel.kernel() #left kernel of rel = orthogonal complement of the image
            if Show_Progress: print(f"..................done! (dimension {H.dimension()})***",flush = True)
            del rel #don't need differential anymore
            B = H.basis_matrix().gram_schmidt()[0]
            del H
            # the orthogonal basis for H is also perpendicular to the image of the differential
            # so can compute trace of doubling matrix on homology by summing over <Doubler * b_i, b_i> / <b_i,b_i> over the basis
            # need to use inner product since Doubler might take vectors in H to sum of something in H + something in Im(diff).
            # but since H is orthogonal to Im(diff), the inner product factors through the projection onto H === induced action on homology.

            for evalue in evalues:
                trace = 0
                D = double_matrix(botcells,evalue)
                for vect in B:
                    trace += (vect*D).dot_product(vect) / vect.dot_product(vect)
                del D
                evalue = [2^d for d in evalue]
                if Show_Progress:
                    print(f"Trace of matrix with e-values {evalue} is: {trace}")
                eigenvalues[p] += [(evalue,trace)]
            del B

    return eigenvalues

In [2]:
conf_trace(11, 2, evalues = [[1,0],[1,1]], codim = 1, partitions = [], Show_Progress = True)

For partition  [11]  -
***Computing differential:.................done!***
***Computing cokernel:.....................done! (dimension 6)***
Trace of matrix with e-values [2, 1] is: 63
Trace of matrix with e-values [2, 2] is: 192
For partition  [10, 1]  -
***Computing differential:.................done!***
***Computing cokernel:.....................done! (dimension 0)***
Trace of matrix with e-values [2, 1] is: 0
Trace of matrix with e-values [2, 2] is: 0
For partition  [9, 2]  -
***Computing differential:.................done!***
***Computing cokernel:.....................done! (dimension 22)***
Trace of matrix with e-values [2, 1] is: 132
Trace of matrix with e-values [2, 2] is: 404
For partition  [9, 1, 1]  -
***Computing differential:.................done!***
***Computing cokernel:.....................done! (dimension 27)***
Trace of matrix with e-values [2, 1] is: 219
Trace of matrix with e-values [2, 2] is: 684
For partition  [8, 3]  -
***Computing differential:.................d

KeyboardInterrupt: 